<a href="https://colab.research.google.com/github/LLMUSER/on-device-tests/blob/main/QA_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.4 MB/s eta 0:00:00


In [21]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00


In [29]:
from transformers import DistilBertTokenizer

# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")

# Sample chunk of text
# performing string manipulation

# The original text with multiple line breaks
original_text = """
"In the heart of a bustling metropolis, the city's skyline looms tall and majestic, with towering skyscrapers that seem to touch the heavens. The streets below are a bustling hive of activity, filled with people from all walks of life going about their daily routines. Cars honk and buses hiss as they navigate the labyrinthine roadways, while pedestrians weave their way through the crowds.

Amidst this urban jungle, there are pockets of tranquility. Parks and gardens provide a respite from the concrete and steel, offering lush greenery and the soothing sounds of nature. Cafes and street vendors offer a delightful array of culinary delights, from aromatic coffee to savory street food.

The city is a melting pot of cultures, where diverse communities coexist and thrive. Museums and galleries showcase the rich tapestry of history and art, while festivals and celebrations bring people together in joyous harmony.

At night, the city transforms into a dazzling spectacle, with neon lights illuminating the skyline and the nightlife coming to life. Clubs pulse with music, theaters host captivating performances, and restaurants serve delectable cuisine late into the evening.

As dawn breaks, the city awakens once more, ready to embrace another day of energy and excitement. It is a place of dreams and opportunities, where the possibilities are endless, and where every corner holds a new adventure. This is the city that never sleeps, a vibrant and dynamic mosaic of life and culture."
"""

# Split the text into paragraphs based on double line breaks
paragraphs = original_text.strip().split('\n\n')

# Join the paragraphs with single line breaks to bring them together
formatted_text = '\n'.join(paragraphs)

# Print the formatted text
print(formatted_text)


# Tokenize the text
tokens = tokenizer.tokenize(original_text)
print(len(tokens))
# Calculate the number of characters (including spaces) in the tokens
num_characters = sum(len(token) for token in tokens)
print(len(tokens))
print(f"Number of characters in the chunk: {num_characters}")


"In the heart of a bustling metropolis, the city's skyline looms tall and majestic, with towering skyscrapers that seem to touch the heavens. The streets below are a bustling hive of activity, filled with people from all walks of life going about their daily routines. Cars honk and buses hiss as they navigate the labyrinthine roadways, while pedestrians weave their way through the crowds.
Amidst this urban jungle, there are pockets of tranquility. Parks and gardens provide a respite from the concrete and steel, offering lush greenery and the soothing sounds of nature. Cafes and street vendors offer a delightful array of culinary delights, from aromatic coffee to savory street food.
The city is a melting pot of cultures, where diverse communities coexist and thrive. Museums and galleries showcase the rich tapestry of history and art, while festivals and celebrations bring people together in joyous harmony.
At night, the city transforms into a dazzling spectacle, with neon lights illumin

In [31]:
!pip install langchain

In [44]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")

# Your long piece of text
long_text = "The Amazon rainforest, also known as Amazonia, is a vast and diverse ecosystem that covers much of South America. It is home to a rich variety of flora and fauna, many of which are found nowhere else on Earth. The rainforest plays a crucial role in maintaining global climate stability and is often referred to as the 'lungs of the Earth.' It faces numerous threats, including deforestation and climate change."


# Split the long text into smaller chunks (e.g., paragraphs)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap = 0)
chunks = text_splitter.split_text(long_text)
# List to store extracted answers
answers = []
# print(chunks)
# Iterate through each chunk
for chunk in chunks:
    # Formulate a question based on the content of the chunk
    question = "What is the Amazon rainforest often referred to as, and why is it significant?"
    inputs = tokenizer(question, chunk, return_tensors="pt")
    outputs = model(**inputs)
    # Tokenize the chunk and question
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

# Convert logits to probabilities (optional)
    start_probs = start_logits.split(1,dim=-1)
    end_probs = end_logits.split(1,dim=-1)

# Find the answer span using argmax or more advanced techniques
    start_index = torch.argmax(start_probs)
    end_index = torch.argmax(end_probs)


    # Extract the answer span from the chunk
    answer_span = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0][start_index:end_index+1]))

    # Append the answer to the list
    answers.append(answer_span)

# Aggregate answers from all chunks
final_answer = " ".join(answers)

# Print or use the final_answer
print(final_answer)


TypeError: ignored

In [53]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")

# Your long piece of text
long_text = "The Amazon rainforest, also known as Amazonia, is a vast and diverse ecosystem that covers much of South America. It is home to a rich variety of flora and fauna, many of which are found nowhere else on Earth. The rainforest plays a crucial role in maintaining global climate stability and is often referred to as the 'lungs of the Earth.' It faces numerous threats, including deforestation and climate change."

# Function to split the long text into chunks
def split_long_text_into_chunks(text, max_chunk_length=400):
    chunks = []
    words = text.split()
    current_chunk = []

    for word in words:
        if sum(len(tokenizer.encode(word)) for word in current_chunk) + len(current_chunk) > max_chunk_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
        current_chunk.append(word)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

# List to store extracted answers
answers = []

# Iterate through each chunk
chunks = split_long_text_into_chunks(long_text)

for chunk in chunks:
    # Formulate a question based on the content of the chunk
    question = "What does Amazon rainforest cover ?"

    # Tokenize the chunk and question
    inputs = tokenizer(chunk, question, return_tensors="pt")

    # Make predictions for the start and end positions of the answer span
    outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Get the token positions with the highest probability for the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    # Extract the answer span from the chunk
    answer_span = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0][start_index:end_index+1]))

    # Append the answer to the list
    answers.append(answer_span)

# Aggregate answers from all chunks
final_answer = " ".join(answers)

# Print or use the final_answer
print(final_answer)



Amazon rainforest , also known as Amazonia , is a vast and diverse ecosystem that covers much of South America
